In [ ]:
class peak_motif_scan:
    '''
    Object with feature of a collection of peaks sequence
    
    '''
    def __init__(self, fasta):
        # data members (instance variables)
        self.seqs, _ = self.get_seq(fasta)
    
    def analyse_seqs(self, scanner):
        results = []
        
        for i, s in enumerate(self.seqs):
            frequency, position = scanner(s)
            if frequency > 0:
                results.append([i, frequency, position])
        
        return results
 
    def scan_cann(self, seq):
        '''calculate cannonical motif frequency'''
        motif = "TGACGTCA"
        left = 0 
        right = len(motif)
        frequency = 0 #count the motif occurence
        position = []
        while right <= len(seq):
            subs = seq[left:right]
            if subs == motif:
                frequency += 1
                position.append([left, right])
                #after recording, we move pointer forward
                left = right 
                right = left + len(motif)
            else:
                left+=1
                right+=1
        return frequency, position
        
        
    def scan_var(self, seq):
        '''calculate var motif frequency'''
        import re
        frequency = 0
        position = []
        motif = r'TGA\w{2}TCA'
        for match in re.finditer(motif, seq):
            frequency += 1
            position.append([match.start(), match.end()])
        return frequency, position
    
    def scan_AP1(self, seq):
        '''calculate AP1 motif frequency'''
        import re
        frequency = 0
        position = []
        motif = r'TGAGTCA'
        for match in re.finditer(motif, seq):
            frequency += 1
            position.append([match.start(), match.end()])
        return frequency, position
    
    def scan_repeats(self, s, targets=['TCA', 'TGA'], gap = 5, rep_tol = 4):
        left = 0 
        right = len(targets[0])
        reps = 0 #count repeating occurence
        running_gap = 0 #count accumulative gap when sliding
        res = [] #keep track of all repeats result
        position = []
        #subs = s[left:right+1]

        while right <= len(s):
            subs = s[left:right]
            #when runing gap larger than gap tolerance and repeats larger than 4, we record the event and restart scanning.  
            if running_gap > gap:
                if reps > rep_tol:
                    res.append(reps)
                    end = right
                    position.append([start, end])

                reps = 0
                running_gap = 0
            #scanning repeats and add up occurence
            elif subs in targets:
                if reps == 0:
                    start = left #keep track of position when meet the first occurence
                reps += 1
                #after count one occr, we move pointer forward
                left = right 
                right = left + len(targets[0])
                running_gap = 0 #gap to zero
        
            else:
                left+=1
                right+=1
                running_gap +=1

        return len(res), position

    def get_seq(self, FASTA):
        inFile = open(FASTA,'r')
        headerList = []
        seqList = []
        currentSeq = ''
        for line in inFile:
            if line[0] == ">":
                headerList.append(line[1:].strip())
                if currentSeq != '':
                    seqList.append(currentSeq)

                currentSeq = ''
            else:
                currentSeq += line.strip().upper()

        seqList.append(currentSeq)


        return seqList, headerList

In [ ]:
#load peaks
import pandas as pd
peaks = pd.read_csv("/Volumes/kjlab/Ben/mouse_ccs/Ben_files/V5_n5_FINAL.bed", sep = "\t", header=None)
#get fasta
!bedtools getfasta -fi /Users/KStraessler/Data_KJ/LL/ASPS/Mm10/chrAllNormal.fasta -bed \
/Volumes/kjlab/Ben/mouse_ccs/Ben_files/V5_n5_FINAL.bed -fo mCCS_v5.fa
# create object of a class with mCCS_V5 sequence
V5_scan = peak_motif_scan("mCCS_v5.fa")

## now it is time to scan for different motifs by calling different scanner

In [ ]:
#scan for variant motif and save the reuslt
results = V5_scan.analyse_seqs(V5_scan.scan_var) #(scan_repeats, scan_AP1, scan_cann)
rep_peaks = [r[0] for r in results]
res = [0]*len(peaks)
cnt = 0
for i, r in enumerate(res):
    if i in rep_peaks:
        res[i] = results[cnt][1]
        cnt+=1
#replace original peak score with new motif score
peaks[4] = res
#select peaks with at least 1 motif
cann_peaks = peaks[peaks[4] > 0] 
#save file 
cann_peaks.to_csv('var_V5Peaks.bed', sep = '\t', header=None, index=None)